<a href="https://colab.research.google.com/github/geekysydney/code-unza15-CSC-4792-project-team-15-repository-for-project-team-15/blob/main/code_unza25_csc4792_project_team_15_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Business Understanding

